In [ ]:
!pip install ipykernel ipython pylint mypy nltk black

In [ ]:

import time

start = time.time()

In [ ]:
import json
import os
import sys
import typing as T

!pip install python-dotenv pandas

import dotenv
import pandas as pd

dotenv.load_dotenv()

# Secrets
GOOGLE_API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
assert GOOGLE_API_KEY is not None, "GOOGLE_PLACES_API_KEY is not set in the `.env` file"
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
assert OPEN_AI_API_KEY is not None, "OPEN_AI_API_KEY is not set in the `.env` file"
MAPBOX_API_KEY = os.getenv("MAPBOX_API_KEY")
assert MAPBOX_API_KEY is not None, "MAPBOX_API_KEY is not set in the `.env` file"

CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)
SRC_DIR = os.path.join(ROOT_DIR, "src")

sys.path.append(SRC_DIR)

UI_INPUT = False


In [ ]:
# Fields to be returned by detailed search
DEFAULT_FIELDS = [
    "places.id",
    "places.formattedAddress",
    "places.displayName",
    "places.location",
    "places.rating",
    "places.googleMapsUri",
    "places.websiteUri",
    "places.businessStatus",
    "places.priceLevel",
    "places.userRatingCount",
    "places.primaryType",
    "places.types",
    "places.editorialSummary",
    "places.goodForChildren",
]

MIN_RATING = 3.5
MIN_RATING_COUNT = 100


In [ ]:
!pip install langchain langchain-core

from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# these columns should match the members of the Itinerary class
DAY_COLUMN = "day"
ACTIVITY_TYPE_COLUMN = "activity_type"
LOCATION_COLUMN = "location"
DESCRIPTION_COLUMN = "description"


class Itinerary(BaseModel):
    day: T.List[str] = Field(
        description="a list that indicates the day of the corresponding activity"
    )
    activity_type: T.List[str] = Field(
        description="a list of the activities type of the itinerary like breakfast, dinner etc"
    )
    location: T.List[str] = Field(
        description=(
            "a list of the location that existis in "
            "google places of the corresponding activity type"
        )
    )
    description: T.List[str] = Field(
        description="a brief description of the activity. The description must be 2-3 word phrases"
    )


ITINERARY_PROMPT_TEMPLATE = PromptTemplate(
    template="""Given the user's input below:
Location: {location}
Number of People: {number_of_people}
Date: {date}
Duration (Days): {duration_days}
Tell Us About Your Group: {group_type}
Briefly describe the trip you envision including the groups interests in activities, food etc: {description}

Create an itinerary providing the activities for breakfast, morning activity, lunch, afternoon
activity, dinner and evening activity, the location which must exist in google places and a
brief description of the activity or the place

Make sure to include all the itinerary days""",
    input_variables=[
        "location",
        "number_of_people",
        "date",
        "duration",
        "group_type",
        "description",
    ],
)


In [ ]:
inputs = {
    "location": "South Beach Miami, FL",
    "number_of_people": 6,
    "date": "November 2026",
    "duration_days": 3,
    "group_type": "bachelorette party",
    "description": (
        "include boutique hotel options must 4 stars higher onsite spa beach clubs djs"
        "day high end nightclubs list least six restaurant options dinner include least "
        "one nice steakhouse one nice sushi restaurant"
    ),
}

In [ ]:
if UI_INPUT:
    ##############################################################################
    # Skip this block if you want to use the hard coded inputs
    ##############################################################################
    from input_ui import SimpleInputForm

    form = SimpleInputForm()
    form.setup()
    inputs = form.run_form()

In [ ]:
from langchain_core.pydantic_v1 import BaseModel


def get_model_fields(model: BaseModel):
    fields = {}
    for field_name, field_info in model.__fields__.items():
        fields[field_name] = {
            "type": field_info.type_,
            "description": field_info.field_info.description,
        }
    return fields


def calculate_tokens(data: T.Dict[str, str], output: T.Dict[str, str]) -> T.Tuple[int, int, int]:
    input_tokens = sum(len(str(value).split()) for value in data.values())
    output_tokens = sum(len(str(value).split()) for value in output.values())
    total_tokens = input_tokens + output_tokens

    return input_tokens, output_tokens, total_tokens


In [ ]:
!pip install langchain-text-splitters langsmith langchain-openai langchain-community
!pip install pydantic

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from pydantic.v1.types import SecretStr


class OpenAiSearch:
    MODEL = "gpt-3.5-turbo-0125"

    def __init__(self, api_key: SecretStr, verbose: bool = False):
        self.parser = JsonOutputFunctionsParser()
        self.api_key = api_key
        self.verbose = verbose

    def search(
        self,
        inputs: T.Dict[str, str],
        prompt: PromptTemplate,
        model_function: type[BaseModel],
    ) -> T.Any:
        model = ChatOpenAI(api_key=self.api_key, temperature=0, model=self.MODEL)

        openai_functions = [convert_to_openai_function(model_function)]

        chain = prompt | model.bind(functions=openai_functions) | self.parser
        output = chain.invoke(inputs)

        if self.verbose:
            print(output)

        if not output:
            raise ValueError("No output was generated")

        return output

    def calculate_tokens(
        self, inputs: T.Dict[str, str], output: T.Dict[str, str]
    ) -> T.Tuple[int, int, int]:
        input_tokens, output_tokens, total_tokens = calculate_tokens(inputs, output)

        if self.verbose:
            print(f"Input Tokens: {input_tokens}")
            print(f"Output Tokens: {output_tokens}")
            print(f"Total Tokens: {total_tokens}")

        return input_tokens, output_tokens, total_tokens


In [ ]:
llm = OpenAiSearch(OPEN_AI_API_KEY)
output = llm.search(inputs, ITINERARY_PROMPT_TEMPLATE, Itinerary)

print(output)

input_tokens, output_tokens, total_tokens = llm.calculate_tokens(inputs, output)
print(f"Input Tokens: {input_tokens}")
print(f"Output Tokens: {output_tokens}")
print(f"Total Tokens: {total_tokens}")

In [ ]:
# Extract the itinerary content from the response
df = pd.DataFrame(output)

print(df)

In [ ]:
import typing as T

!pip install geopy
from geopy.geocoders import Nominatim

METERS_PER_MILE = 1609.34
METERS_PER_KILOMETER = 1000.0

ALL_FIELDS = [
    "places.formattedAddress",
    "places.displayName",
    "places.nationalPhoneNumber",
    "places.location",
    "places.rating",
    "places.googleMapsUri",
    "places.websiteUri",
    "places.regularOpeningHours",
    "places.businessStatus",
    "places.priceLevel",
    "places.userRatingCount",
    "places.takeout",
    "places.delivery",
    "places.dineIn",
    "places.servesBreakfast",
    "places.primaryTypeDisplayName",
    "places.primaryType",
    "places.editorialSummary",
    "places.outdoorSeating",
    "places.servesCoffee",
    "places.paymentOptions",
    "places.accessibilityOptions",
]

TYPES = [
    "restaurant",  # default
    "bakery",
    "sandwich_shop",
    "coffee_shop",
    "cafe",
    "fast_food_restaurant",
    "store",
    "food",
    "point_of_interest",
    "establishment",
]

TABLE_A_TYPES = [
    "car_dealer",
    "car_rental",
    "car_repair",
    "car_wash",
    "electric_vehicle_charging_station",
    "gas_station",
    "parking",
    "rest_stop",
    "farm",
    "art_gallery",
    "museum",
    "performing_arts_theater",
    "library",
    "preschool",
    "primary_school	school",
    "secondary_school",
    "university",
    "amusement_center",
    "amusement_park",
    "aquarium",
    "banquet_hall",
    "bowling_alley",
    "casino",
    "community_center",
    "convention_center",
    "cultural_center",
    "dog_park",
    "event_venue",
    "hiking_area",
    "historical_landmark",
    "marina",
    "movie_rental",
    "movie_theater",
    "national_park",
    "night_club",
    "park",
    "tourist_attraction",
    "visitor_center",
    "wedding_venue",
    "zoo",
    "accounting",
    "atm",
    "bank",
    "american_restaurant",
    "bakery",
    "bar",
    "barbecue_restaurant",
    "brazilian_restaurant",
    "breakfast_restaurant",
    "brunch_restaurant",
    "cafe",
    "chinese_restaurant",
    "coffee_shop",
    "fast_food_restaurant",
    "french_restaurant",
    "greek_restaurant",
    "hamburger_restaurant",
    "ice_cream_shop",
    "indian_restaurant",
    "indonesian_restaurant",
    "italian_restaurant",
    "japanese_restaurant",
    "korean_restaurant	lebanese_restaurant",
    "meal_delivery",
    "meal_takeaway",
    "mediterranean_restaurant",
    "mexican_restaurant",
    "middle_eastern_restaurant",
    "pizza_restaurant",
    "ramen_restaurant",
    "restaurant",
    "sandwich_shop",
    "seafood_restaurant",
    "spanish_restaurant",
    "steak_house",
    "sushi_restaurant",
    "thai_restaurant",
    "turkish_restaurant",
    "vegan_restaurant",
    "vegetarian_restaurant",
    "vietnamese_restaurant",
    "administrative_area_level_1",
    "administrative_area_level_2",
    "country	locality",
    "postal_code",
    "school_district",
    "city_hall",
    "courthouse",
    "embassy",
    "fire_station	",
    "local_government_office",
    "police",
    "post_office",
    "dental_clinic",
    "dentist",
    "doctor",
    "drugstore",
    "hospital	medical_lab",
    "pharmacy",
    "physiotherapist",
    "spa",
    "bed_and_breakfast",
    "campground",
    "camping_cabin",
    "cottage",
    "extended_stay_hotel",
    "farmstay",
    "guest_house	hostel",
    "hotel",
    "lodging",
    "motel",
    "private_guest_room",
    "resort_hotel",
    "rv_park",
    "church",
    "hindu_temple",
    "mosque",
    "synagogue",
    "barber_shop",
    "beauty_salon",
    "cemetery",
    "child_care_agency",
    "consultant",
    "courier_service",
    "electrician",
    "florist",
    "funeral_home",
    "hair_care",
    "hair_salon",
    "insurance_agency	",
    "laundry",
    "lawyer",
    "locksmith",
    "moving_company",
    "painter",
    "plumber",
    "real_estate_agency",
    "roofing_contractor",
    "storage",
    "tailor",
    "telecommunications_service_provider",
    "travel_agency",
    "veterinary_care",
    "auto_parts_store",
    "bicycle_store",
    "book_store",
    "cell_phone_store",
    "clothing_store",
    "convenience_store",
    "department_store",
    "discount_store",
    "electronics_store",
    "furniture_store",
    "gift_shop",
    "grocery_store",
    "hardware_store",
    "home_goods_store	home_improvement_store",
    "jewelry_store",
    "liquor_store",
    "market",
    "pet_store",
    "shoe_store",
    "shopping_mall",
    "sporting_goods_store",
    "store",
    "supermarket",
    "wholesaler",
    "athletic_field",
    "fitness_center",
    "golf_course",
    "gym",
    "playground",
    "ski_resort",
    "sports_club",
    "sports_complex",
    "stadium",
    "swimming_pool",
    "airport",
    "bus_station",
    "bus_stop",
    "ferry_terminal",
    "heliport",
    "light_rail_station",
    "park_and_ride	subway_station",
    "taxi_stand",
    "train_station",
    "transit_depot",
    "transit_station",
    "truck_stop",
]

DEFAULT_TYPE = TYPES[0]


class Coordinates(T.TypedDict):
    lat: float
    lng: float


class Viewport(T.TypedDict):
    low: Coordinates
    high: Coordinates


class SearchGrid(T.TypedDict):
    center: Coordinates
    viewport: Viewport
    width_meters: float


def meters_to_miles(meters: float) -> float:
    return meters / METERS_PER_MILE


def miles_to_meters(miles: float) -> float:
    return miles * METERS_PER_MILE


def extract_city(address: str) -> T.Optional[str]:
    """
    Given an address, extract the city from it.
    The address is expected to be a comma-separated string.
    Example:
        1340, Saint Nicholas Avenue, Washington Heights, Manhattan Community Board 12,
        Manhattan, City of New York, New York County, New York, 10033, United States
    """
    parts = address.split(",")
    city = None
    zip_code_index: T.Optional[int] = None

    print(address)
    for i, part in enumerate(parts):
        part = part.strip()
        if "City of " in part:
            city = part[len("City of ") :].strip()
            break
        if part.isnumeric() and len(part) == 5 and i > 1:
            zip_code_index = i

    if not city and zip_code_index:
        # If none of the recognizable keywords are found,
        # assume city is 3 parts before the zip code
        city = parts[zip_code_index - 3].strip() if zip_code_index - 3 >= 0 else None

        if city is None:
            return None

        if "County" in city:
            return None

        if any(char.isdigit() for char in city):
            return None

    return city


def get_city_center_coordinates(city_name: str) -> T.Optional[Coordinates]:
    # Initialize the Nominatim geocoder
    geolocator = Nominatim(user_agent="tgtg")

    # Use the geocoder to geocode the city name
    location = geolocator.geocode(city_name)

    if not location:
        return None

    return Coordinates(lat=location.latitude, lng=location.longitude)


In [ ]:
import copy
import json
import os
import typing as T

import requests


def call_api(
    url: str,
    headers: T.Optional[T.Dict[str, T.Any]] = None,
    params: T.Optional[T.Dict[str, T.Any]] = None,
    json_data: T.Optional[T.Dict[str, T.Any]] = None,
    timeout: float = 10.0,
) -> T.Dict[T.Any, T.Any]:
    headers = headers or {}
    json_data = json_data or {}
    params = params or {}

    try:
        response = requests.post(
            url, headers=headers, params=params, json=json_data, timeout=timeout
        ).json()
        print(url, headers, params, json_data)
        if not isinstance(response, dict):
            print(f"Failed results from {url}")
            print(response)
            return {}

        return response
    except Exception as exception:  # pylint: disable=broad-except
        print(f"Failed results for {url}")
        print(exception)
        raise exception


class GoogleMapsAPI:
    def __init__(self, api_key: str, verbose: bool = False) -> None:
        self.api_key = api_key
        self.base_url = "https://maps.googleapis.com/maps/api"
        self.verbose = verbose

    def find_place_from_location(self, place: str, location: Coordinates) -> T.Dict[T.Any, T.Any]:
        location_string = f"{location['lat']},{location['lng']}"
        params = {
            "input": place,
            "inputtype": "textquery",
            "fields": "name,place_id",
            "locationbias": f"point:{location_string}",
            "key": self.api_key,
        }

        url = os.path.join(self.base_url, "place", "findplacefromtext", "json")

        if self.verbose:
            print(f"Searching for {place} at {location_string}")

        return call_api(url, params=params)

    def nearby_search(
        self,
        location: Coordinates,
        radius_meters: int,
        location_type: T.List[str],
        keyword: str,
    ) -> T.Dict[T.Any, T.Any]:
        location_string = f"{location['lat']},{location['lng']}"
        assert location_type in TYPES, f"Invalid location type {location_type}"

        params = {
            "location": location_string,
            "radius": radius_meters,
            "type": location_type,
            "keyword": keyword,
            "key": self.api_key,
        }

        url = os.path.join(self.base_url, "place", "nearbysearch", "json")

        if self.verbose:
            print(f"Searching for {keyword} at {location_string}")

        return call_api(url, params=params)

    def details_from_place_id(
        self, place_id: str, fields: T.Optional[T.List[str]] = None
    ) -> T.Dict[T.Any, T.Any]:
        fields = fields or [field.replace("places.", "") for field in DEFAULT_FIELDS]

        params = {
            "place_id": place_id,
            "fields": ",".join(fields),
            "key": self.api_key,
        }

        url = os.path.join(self.base_url, "place", "details", "json")

        if self.verbose:
            print(f"Getting details for {place_id}")

        return call_api(url, params=params)


class GooglePlacesAPI:
    HEADERS = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": "",
        "X-Goog-FieldMask": "",
    }
    BASIC_FIELDS = "places.formattedAddress,places.displayName"
    MIN_VIEWPOINT_WIDTH_METERS = 100.0
    MAX_VIEWPOINT_WIDTH_METERS = 700.0
    VIEWPOINT_WIDTH_STEP_METERS = 50.0

    def __init__(self, api_key: str, verbose: bool = False) -> None:
        self.api_key = api_key
        self.HEADERS["X-Goog-Api-Key"] = api_key
        self.base_url = "https://places.googleapis.com/v1"
        self.verbose = verbose

    def text_search(
        self,
        query: str,
        fields: T.Optional[T.List[str]] = None,
        data: T.Optional[T.Dict[str, T.Any]] = None,
    ) -> T.Dict[T.Any, T.Any]:
        json_data = {
            "textQuery": query,
        }

        if data:
            json_data.update(data)

        if self.verbose:
            print(f"Searching for {query}")
            print(f"Data: {json.dumps(json_data, indent=2)}")

        headers = copy.deepcopy(self.HEADERS)

        headers["X-Goog-FieldMask"] = ",".join(fields) if fields else self.BASIC_FIELDS

        url = os.path.join(self.base_url, "places:searchText")

        return call_api(url, headers, json_data)

    def nearby_places(
        self,
        latitude: float,
        longitude: float,
        radius_meters: float,
        fields: T.Optional[T.List[str]] = None,
        data: T.Optional[T.Dict[str, T.Any]] = None,
    ) -> T.Dict[T.Any, T.Any]:
        radius_meters = min(radius_meters, 50000.0)

        if fields is None:
            fields = DEFAULT_FIELDS

        headers = copy.deepcopy(self.HEADERS)

        headers["X-Goog-FieldMask"] = ",".join(fields) if fields else self.BASIC_FIELDS

        json_data: T.Dict[str, T.Any] = {
            "locationRestriction": {
                "circle": {
                    "center": {"latitude": latitude, "longitude": longitude},
                    "radius": radius_meters,
                }
            },
        }

        if data:
            json_data.update(data)

        url = os.path.join(self.base_url, "places:searchNearby")

        if self.verbose:
            print(
                f"Searching for nearby places to {latitude}, {longitude} "
                f"within {radius_meters} meters"
            )
        if self.verbose:
            print(f"{json.dumps(json_data, indent=2)}")

        return call_api(url, json_data=json_data, headers=headers)

    def search_location_radius(
        self,
        latitude: float,
        longitude: float,
        radius_meters: float,
        query: str,
        fields: T.Optional[T.List[str]] = None,
        included_type: T.Optional[str] = None,
        data: T.Optional[T.Dict[str, T.Any]] = None,
    ) -> T.Dict[T.Any, T.Any]:
        radius_meters = min(radius_meters, 50000.0)

        if self.verbose:
            print(f"Searching for {query} within {radius_meters} meters of {latitude}, {longitude}")
        json_data: T.Dict[str, T.Any] = {
            "locationBias": {
                "circle": {
                    "center": {"latitude": latitude, "longitude": longitude},
                    "radius": radius_meters,
                }
            },
        }

        if data:
            json_data.update(data)

        if included_type is not None:
            json_data["includedType"] = included_type

        if self.verbose:
            print(f"{json.dumps(json_data, indent=2)}")
        return self.text_search(query=query, fields=fields, data=json_data)

In [ ]:

import concurrent.futures
import threading
import typing as T
from threading import Lock

ItineraryPlaceDetailsType = T.List[T.Tuple[str, T.Dict[str, T.List[T.Any]]]]
NearbyPlaceDetailsType = T.Dict[str, T.List[T.Dict[str, T.Any]]]


class SearchPlaces:

    def __init__(self, api_key: str, verbose: bool = False):
        self.api_key = api_key
        self.verbose = verbose
        self.itinerary_place_details: ItineraryPlaceDetailsType = []
        self.nearby_place_details: NearbyPlaceDetailsType = {}
        self.total_api_calls = {
            "places": 0,
            "maps": 0,
        }

        self.lock = Lock()

    @staticmethod
    def is_acceptable_location(
        original: T.Dict[str, T.Any],
        proposed: T.Dict[str, T.Any],
        compare_types: bool = False,
        verbose: bool = False,
    ) -> bool:
        rating = proposed.get("rating", 0.0)
        if float(rating) < MIN_RATING:
            if verbose:
                print(f"Rating is too low: {rating}")
            return False

        rating_count = proposed.get("userRatingCount", 0)
        if int(rating_count) < MIN_RATING_COUNT:
            if verbose:
                print(f"Rating count is too low: {rating_count}")
            return False

        if proposed.get("businessStatus") != "OPERATIONAL":
            if verbose:
                print(f"Business status is not operational: {proposed.get('businessStatus')}")
            return False

        if compare_types and proposed.get("primaryType") not in original.get("types", []):
            if verbose:
                print(
                    f"Primary type {proposed.get('primaryType')} "
                    f"is not in original types {original.get('types', [])}"
                )
            return False

        if proposed.get("id") == original.get("id"):
            if verbose:
                print(f"ID is the same: {proposed.get('id')}")
            return False

        return True

    @staticmethod
    def call_api(gmap_func: T.Callable, *args: T.Any, **kwargs: T.Any) -> T.Any:
        try:
            print(f"Calling Google Maps API {gmap_func} with {args} and {kwargs}")
            result = gmap_func(*args, **kwargs)
        except Exception as exc:  # pylint: disable=broad-except
            print(f"Error calling Google Maps API: {exc}")
            return None

        if not result or result.get("status") != "OK" or not result.get("results"):
            if not result:
                print(f"Unable to {gmap_func} to get api info")
            else:
                print(f"Unable to {gmap_func} to get api info status: {result.get('status')}")
            return None

        return result

    @staticmethod
    def _get_place_details(
        api_key: str,
        itinerary_info: T.Tuple[str, str, str],
        city_name: str,
        radius_meters: int,
        itinerary_place_details: ItineraryPlaceDetailsType,
        nearby_place_details: NearbyPlaceDetailsType,
        lock: threading.Lock,
        verbose: bool = False,
    ) -> T.Dict[str, int]:
        total_api_calls = {
            "places": 0,
            "maps": 0,
        }

        location_name, description, activity_type = itinerary_info

        for query in [
            f"{location_name} in {city_name}",
            f"{activity_type} at {description} in {city_name}",
        ]:
            gplaces = GooglePlacesAPI(api_key, verbose=False)
            result = gplaces.text_search(
                query=query,
                fields=DEFAULT_FIELDS,
                data={
                    "minRating": MIN_RATING,
                },
            )

            total_api_calls["places"] += 1

            if result and len(result.get("places", [])) > 0:
                break

        if not result or len(result.get("places", [])) == 0:
            print(f"No places found for {location_name}")

        place_result = result["places"][0]

        with lock:
            itinerary_place_details.append((location_name, place_result))

        store_types = [t for t in place_result.get("types", [DEFAULT_TYPE]) if t in TABLE_A_TYPES]

        print(
            f"Getting nearby places for {location_name} at "
            f"{place_result['location']} with types {store_types}"
        )

        data = {
            "minRating": MIN_RATING,
        }

        if store_types:
            data["includedTypes"] = store_types

        nearby_places = gplaces.nearby_places(
            latitude=place_result["location"]["latitude"],
            longitude=place_result["location"]["longitude"],
            radius_meters=radius_meters,
            fields=DEFAULT_FIELDS,
            data=data,
        )
        total_api_calls["places"] += 1

        if not nearby_places or len(nearby_places.get("places", [])) == 0:
            print(f"Unable to get nearby places for {location_name}")
            return total_api_calls

        nearby_place_details[location_name] = []
        for nearby_result in nearby_places["places"]:
            if not SearchPlaces.is_acceptable_location(
                place_result, nearby_result, verbose=verbose
            ):
                print(f"Skipping {nearby_result['displayName']['text']} as it is not acceptable")
                continue

            with lock:
                nearby_place_details[location_name].append(nearby_result)

        nearby_list = nearby_place_details[location_name]

        primary_type = place_result.get("primaryType")

        sorted_nearby_list = [
            item for item in nearby_list if item.get("primaryType") == primary_type
        ] + [item for item in nearby_list if item.get("primaryType") != primary_type]

        nearby_place_details[location_name] = sorted_nearby_list

        print(
            (i["description"]["text"], i["primaryType"])
            for i in nearby_place_details[location_name]
        )

        print(f"Found {len(nearby_place_details[location_name])} nearby places for {location_name}")

        return total_api_calls

    def search(
        self,
        city: str,
        itinerary: T.Dict[str, T.List[str]],
        radius_meters: int = 1500,
        single_thread: bool = False,
    ) -> T.Tuple[
        ItineraryPlaceDetailsType,
        NearbyPlaceDetailsType,
        Coordinates,
        int,
    ]:
        city_coordinates = get_city_center_coordinates(city)

        if not city_coordinates:
            raise ValueError(f"Unable to get coordinates for city: {city}")

        print(f"{city} coordinates: {city_coordinates}")

        self.total_api_calls = {
            "places": 0,
            "maps": 0,
        }

        if single_thread:
            for index in range(len(itinerary[LOCATION_COLUMN])):
                itinerary_info = (
                    itinerary[LOCATION_COLUMN][index],
                    itinerary[DESCRIPTION_COLUMN][index],
                    itinerary[ACTIVITY_TYPE_COLUMN][index],
                )
                api_calls = self._get_place_details(
                    self.api_key,
                    itinerary_info,
                    city,
                    radius_meters,
                    self.itinerary_place_details,
                    self.nearby_place_details,
                    self.lock,
                    verbose=self.verbose,
                )
                for key, value in api_calls.items():
                    self.total_api_calls[key] += value
        else:
            with concurrent.futures.ThreadPoolExecutor() as executor:
                futures = [
                    executor.submit(
                        self._get_place_details,
                        self.api_key,
                        (
                            itinerary[LOCATION_COLUMN][index],
                            itinerary[DESCRIPTION_COLUMN][index],
                            itinerary[ACTIVITY_TYPE_COLUMN][index],
                        ),
                        city,
                        radius_meters,
                        self.itinerary_place_details,
                        self.nearby_place_details,
                        self.lock,
                    )
                    for index in range(len(itinerary[LOCATION_COLUMN]))
                ]
                concurrent.futures.wait(futures)

                for future in futures:
                    for key, value in future.result().items():
                        self.total_api_calls[key] += value

        return (
            self.itinerary_place_details,
            self.nearby_place_details,
            city_coordinates,
            sum(self.total_api_calls.values()),
        )


In [ ]:
# Inputs for the nearby lookup, if store_type is left as None, it will
# default to match the type of the place that we are searching nearby from
MAX_NEARBY_PLACES = 2
radius_meters = 1500

In [ ]:
search = SearchPlaces(GOOGLE_API_KEY, verbose=True)
itinerary_place_details, nearby_place_details, city_coordinates, total_api_calls = search.search(
    city=inputs["location"],
    itinerary=output,
    radius_meters=radius_meters,
    single_thread=False,
)

In [ ]:
df["place_id"] = [item["id"] for _, item in itinerary_place_details]
print(df)

print(f"Num results: {len(itinerary_place_details)}")
print(f"Total API calls: {total_api_calls}")

In [ ]:
for place, item in itinerary_place_details:
    main_type = item.get("primaryType", item.get("types", ["UNKNOWN"])[0])
    print(f"{item['displayName']['text']}: {main_type}")
    nearby_names = [
        f"{i['displayName']['text']}: {i.get('primaryType', 'UNKNOWN')}"
        for i in nearby_place_details.get(place, [])
    ]
    for name in nearby_names:
        print(f"  - {name}")

with open(os.path.join(ROOT_DIR, "places.json"), "w", encoding="utf-8") as outfile:
    json.dump({"results": itinerary_place_details}, outfile, ensure_ascii=True, indent=4)
with open(os.path.join(ROOT_DIR, "nearby_places.json"), "w", encoding="utf-8") as outfile:
    json.dump(nearby_place_details, outfile, ensure_ascii=True, indent=4)

In [ ]:
total_time = time.time() - start

print(f"Total time taken: {total_time:.2f} seconds")

In [ ]:
!pip install plotly numpy nb-clean

import plotly.express as px
import plotly.graph_objects as go

assert len(itinerary_place_details) != 0, "No places found in the itinerary"
assert len(nearby_place_details) != 0, "No nearby places found"

# Create a map showing the places in the itinerary and nearby places
print("Creating map...")
px.set_mapbox_access_token(MAPBOX_API_KEY)

places_dfs = {}
color = 0
color_increment = 1.0 / len(itinerary_place_details)
for name, place in itinerary_place_details:
    color += color_increment
    rows = [(place["displayName"]["text"], *place["location"].values(), color)]
    max_clamp = min(MAX_NEARBY_PLACES, len(nearby_place_details.get(name, [])))
    rows.extend(
        [
            (x["displayName"]["text"], *x["location"].values(), color)
            for x in nearby_place_details.get(name, [])[:max_clamp]
        ]
    )
    places_dfs[name] = pd.DataFrame(rows, columns=["name", "latitude", "longitude", "color"])


fig = go.Figure()

for row in df.iterrows():
    places_coords_df = places_dfs[row[1][LOCATION_COLUMN]]
    trace_name = (
        f"Day {row[1][DAY_COLUMN]}: {row[1][ACTIVITY_TYPE_COLUMN]} at {row[1][LOCATION_COLUMN]}"
    )
    fig.add_trace(
        go.Scattermapbox(
            lat=places_coords_df["latitude"],
            lon=places_coords_df["longitude"],
            mode="markers",
            marker=go.scattermapbox.Marker(
                size=5, color=places_coords_df["color"], colorscale="Rainbow", cmin=0, cmax=1
            ),
            text=places_coords_df["name"],
            name=trace_name,
        )
    )

fig.update_layout(
    autosize=True,
    hovermode="closest",
    mapbox=go.layout.Mapbox(
        accesstoken=os.getenv("MAPBOX_API_KEY"),
        bearing=0,
        center=go.layout.mapbox.Center(lat=city_coordinates["lat"], lon=city_coordinates["lng"]),
        pitch=0,
        zoom=12,
    ),
    height=800,
    width=1000,
    title_text=f"Itinerary for `{inputs['group_type']}` in {inputs['location']}",
    title_x=0.5,
    annotations=[
        dict(
            text=(f"Nearby {MAX_NEARBY_PLACES} places are shown for each itinerary location. "),
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.0,
            xanchor="center",
            yanchor="top",
            font=dict(size=14, color="black"),
        )
    ],
)

fig.show()